In [2]:
#activate conda env AutencoderTF env tf '2.10.0' , python 3.9.16
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import torch
import sys
print(sys.path)
sys.path.append('c:\\Users\\buehl\\Dropbox\\ZHAW\\MSE_DataScience\\23FS\\VT2\\Anomaly_detection_images\\bosch_AD\\LatentOE-AD')
from loader.LoadData import CIFAR10_feat , FMNIST_feat

import tensorflow as tf

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, precision_score, recall_score 
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


print(f'tensorflow_version:{tf.__version__}')
print(f'torch_version:{torch.__version__}')

['c:\\Users\\buehl\\Dropbox\\ZHAW\\MSE_DataScience\\23FS\\VT2\\Anomaly_detection_images\\bosch_AD\\LatentOE-AD\\Additional_Code_BULE', 'c:\\Users\\buehl\\git\\projects\\LatentOE-AD', 'c:\\Users\\buehl\\anaconda3\\envs\\AutencoderTF\\python39.zip', 'c:\\Users\\buehl\\anaconda3\\envs\\AutencoderTF\\DLLs', 'c:\\Users\\buehl\\anaconda3\\envs\\AutencoderTF\\lib', 'c:\\Users\\buehl\\anaconda3\\envs\\AutencoderTF', '', 'c:\\Users\\buehl\\anaconda3\\envs\\AutencoderTF\\lib\\site-packages', 'c:\\Users\\buehl\\anaconda3\\envs\\AutencoderTF\\lib\\site-packages\\win32', 'c:\\Users\\buehl\\anaconda3\\envs\\AutencoderTF\\lib\\site-packages\\win32\\lib', 'c:\\Users\\buehl\\anaconda3\\envs\\AutencoderTF\\lib\\site-packages\\Pythonwin']
tensorflow_version:2.10.0
torch_version:2.0.1


In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.



print (x_train.shape)
print (x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
latent_dim = 64 

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28, 28))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
dot_img_file = 'C:/Users/buehl/OneDrive/Desktop/model_1.png'
tf.keras.utils.plot_model(autoencoder, to_file=dot_img_file, show_shapes=True)

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[test_mask][i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

In [5]:
#np.random.seed(42)
from skimage.metrics import mean_squared_error, normalized_root_mse,normalized_mutual_information

pmi,mse,nrmse,label,anomaly=[],[],[],[],[]

for i in list(set(y_train)):
    autoencoder = Autoencoder(latent_dim)
    autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
    train_mask=(y_train==i)
    autoencoder.fit(x_train[train_mask], x_train[train_mask],epochs=10,shuffle=True)
    
    #testing normal
    test_mask=(y_test==i)
    encoded_imgs = autoencoder.encoder(x_test[test_mask]).numpy()
    decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

    for j in range(0,len(encoded_imgs)):
        mse.append(mean_squared_error(x_test[test_mask][j],decoded_imgs[j]))
        nrmse.append(normalized_root_mse(x_test[test_mask][j],decoded_imgs[j]))
        pmi.append(normalized_mutual_information(x_test[test_mask][j],decoded_imgs[j])-1)
        label.append(i)
        anomaly.append(0)
    
    #testing anomalies
    test_mask_anomaly=(y_test!=i)
    random_indexes = np.random.choice(range(0,len(x_test[test_mask_anomaly])), size=int(0.1*len(test_mask)), replace=False)
    anomalies=x_test[test_mask_anomaly][random_indexes] 

    encoded_imgs_a = autoencoder.encoder(anomalies).numpy()
    decoded_imgs_a = autoencoder.decoder(encoded_imgs_a).numpy()

    for j in range(0,len(encoded_imgs_a)):
        mse.append(mean_squared_error(anomalies[j],decoded_imgs_a[j]))
        nrmse.append(normalized_root_mse(anomalies[j],decoded_imgs_a[j]))
        pmi.append(normalized_mutual_information(anomalies[j],decoded_imgs_a[j])-1)
        label.append(i)
        anomaly.append(1)

Epoch 1/10
188/188 [==============================] - 1s 2ms/step - loss: 0.0485
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0256
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0199
Epoch 4/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 5/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0147
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0133
Epoch 7/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0122
Epoch 8/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0113
Epoch 9/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0106
Epoch 10/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0101
Epoch 1/10
188/188 [==============================] - 1s 3ms/step - loss: 0.0357
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0145
Epoch 3/10
188/188 [=======

In [ ]:
only_clean = pd.DataFrame({'mse_': mse, 'nrmse_': nrmse,'nmi_':pmi,'label_of_normal_': label,'anomaly_':anomaly})#, 
only_clean

In [ ]:
import seaborn as sns
sns.histplot(data=only_clean,x='label_of_normal_',y='mse_',hue='anomaly_').set(title="mse only clean data per class")

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
sns.stripplot(data=only_clean,x='label_of_normal_',y='mse_',hue='anomaly_',palette="deep").set(title="mse only clean data per class")

In [7]:
from skimage.metrics import mean_squared_error, normalized_root_mse,normalized_mutual_information
from sklearn.preprocessing import normalize, MinMaxScaler
root="C:/Users/buehl/Dropbox/ZHAW/MSE_DataScience/23FS/VT2/Anomaly_detection_images/bosch_AD/LatentOE-AD/DATA/fmnist_features/"